### 模板模式 
编写优秀代码的一个要素是避免冗余。在面向对象编程中，方法和函数是我们用来避免编写
冗余代码的重要工具。回想第15章中的sorted()例子。sorted()函数非常通用，可使用任意键
来对多种数据结构（列表、元组和命名元组等）进行排序。这是一个良好函数的定义。 
sorted()这样的函数属于理想的案例。现实中，我们没法始终写出100%通用的代码。许多
算法都有一些（但并非全部）通用步骤。广度优先搜索（Breadth-First Search，BFS）和深度优先
搜索（Depth-First Search，DFS）是其中不错的例子，这两个流行的算法应用于图搜索问题。起
初，我们提出独立实现两个算法 。函数bfs()和dfs()在start和end之间存在
一条路径时返回一个元组(True, path)；如果路径不存在，则返回(False, path)（此时，
path为空）。 

In [1]:
#广度优先搜索
def bfs(graph, start, end):
    path = []
    visited = [start]
    while visited:
        current = visited.pop(0)
        if current not in path:
            path.append(current)
            if current == end:
                print(path)
                return (True, path)
            # 两个顶点若不相连则跳过
            if current not in graph:
                continue
        #两个算法仅这一行不一样
        visited = visited + graph[current]
    return (False, path)

#深度优先
def dfs(graph, start, end):
    path = []
    visited = [start]
    while visited:
        current = visited.pop(0)
        if current not in path:
            path.append(current)
            if current == end:
                print(path)
                return (True, path)
            # 两个顶点若不相连则跳过
            if current not in graph:
                continue
        #两个算法仅这一行不一样
        visited = graph[current] + visited
    return (False, path)


先使用Wikipedia提供的图（请参考网页［t.cn/RqrBp3p］）来测试算法。为了简化，假设该图
是有向的。这意味着只能朝一个方向移动，我们可以检测如何从Frankfurt到Mannheim，而不是另
一个方向。 
可以使用列表的字典结构来表示这个有向图。每个城市是字典中的一个键，列表的内容是从
该城市始发的所有可能目的地。叶子顶点的城市（例如，Erfurt）使用一个空列表即可（无目的
地）。 

In [ ]:
def main():
    graph = {
        'Frankfurt': ['Mannheim', 'Wurzburg', 'Kassel'],
        'Mannheim': ['Karlsruhe'],
        'Karlsruhe': ['Augsburg'],
        'Augsburg': ['Munchen'],
        'Wurzburg': ['Erfurt', 'Nurnberg'],
        'Nurnberg': ['Stuttgart', 'Munchen'],
        'Kassel': ['Munchen'],
        'Erfurt': [],
        'Stuttgart': [],
        'Munchen': []
    }

    bfs_path = bfs(graph, 'Frankfurt', 'Nurnberg')
    dfs_path = dfs(graph, 'Frankfurt', 'Nurnberg')
    print('bfs Frankfurt-Nurnberg: {}'.format(bfs_path[1] if bfs_path[0] else 'Not found'))
    print('dfs Frankfurt-Nurnberg: {}'.format(dfs_path[1] if dfs_path[0] else 'Not found'))

    bfs_nopath = bfs(graph, 'Wurzburg', 'Kassel')
    print('bfs Wurzburg-Kassel: {}'.format(bfs_nopath[1] if bfs_nopath[0] else 'Not found'))
    dfs_nopath = dfs(graph, 'Wurzburg', 'Kassel')
    print('dfs Wurzburg-Kassel: {}'.format(dfs_nopath[1] if dfs_nopath[0] else 'Not found'))


In [2]:
main()

['Frankfurt', 'Mannheim', 'Wurzburg', 'Kassel', 'Karlsruhe', 'Erfurt', 'Nurnberg']
['Frankfurt', 'Mannheim', 'Karlsruhe', 'Augsburg', 'Munchen', 'Wurzburg', 'Erfurt', 'Nurnberg']
bfs Frankfurt-Nurnberg: ['Frankfurt', 'Mannheim', 'Wurzburg', 'Kassel', 'Karlsruhe', 'Erfurt', 'Nurnberg']
dfs Frankfurt-Nurnberg: ['Frankfurt', 'Mannheim', 'Karlsruhe', 'Augsburg', 'Munchen', 'Wurzburg', 'Erfurt', 'Nurnberg']
bfs Wurzburg-Kassel: Not found
dfs Wurzburg-Kassel: Not found


从性质来看，结果并不能表明什么，因为DFS和BFS不能很好地处理加权图（权重完全被忽
略了）。处理加权图更好的算法是（Dijkstra的）最短路径优先算法、Bellman-Ford算法和A*算法
等。然而，我们仍然希望按打算的那样遍历图。我们期望的算法输出是一个城市列表，这些城市
是在搜索从Frankfurt到Nurnberg的路径时访问过的。 
结果看起来没问题。BFS按广度进行遍历，DFS则按深度进行遍历，两个算法都没返回任何
非期望的结果。这样不错，但我们的代码仍然有一个问题，那就是冗余。两个算法之间仅有一处
不同，但其余代码都写了两遍。对于这个问题我们能做点什么吗？ 

是的！这个问题可以通过模板设计模式（Template design pattern）来解决。这个模式关注的
是消除代码冗余，其思想是我们应该无需改变算法结构就能重新定义一个算法的某些部分。为了
避免重复而进行必要的重构之后，我们来看看代码会变成什么样子。 

In [3]:
def traverse(graph, start, end, action):
    path = []
    visited = [start]
    while visited:
        current = visited.pop(0)
        if current not in path:
            path.append(current)
            if current == end:
                return (True, path)
            # 
            if current not in graph:
                continue
        visited = action(visited, graph[current])
    return (False, path)


def extend_bfs_path(visited, current):
    return visited + current


def extend_dfs_path(visited, current):
    return current + visited


def main():
    graph = {
        'Frankfurt': ['Mannheim', 'Wurzburg', 'Kassel'],
        'Mannheim': ['Karlsruhe'],
        'Karlsruhe': ['Augsburg'],
        'Augsburg': ['Munchen'],
        'Wurzburg': ['Erfurt', 'Nurnberg'],
        'Nurnberg': ['Stuttgart', 'Munchen'],
        'Kassel': ['Munchen'],
        'Erfurt': [],
        'Stuttgart': [],
        'Munchen': []
    }

    bfs_path = traverse(graph, 'Frankfurt', 'Nurnberg', extend_bfs_path)
    dfs_path = traverse(graph, 'Frankfurt', 'Nurnberg', extend_dfs_path)
    print('bfs Frankfurt-Nurnberg: {}'.format(bfs_path[1] if bfs_path[0] else 'Not found'))
    print('dfs Frankfurt-Nurnberg: {}'.format(dfs_path[1] if dfs_path[0] else 'Not found'))

    bfs_nopath = traverse(graph, 'Wurzburg', 'Kassel', extend_bfs_path)
    print('bfs Wurzburg-Kassel: {}'.format(bfs_nopath[1] if bfs_nopath[0] else 'Not found'))
    dfs_nopath = traverse(graph, 'Wurzburg', 'Kassel', extend_dfs_path)
    print('dfs Wurzburg-Kassel: {}'.format(dfs_nopath[1] if dfs_nopath[0] else 'Not found'))


In [4]:
main()

bfs Frankfurt-Nurnberg: ['Frankfurt', 'Mannheim', 'Wurzburg', 'Kassel', 'Karlsruhe', 'Erfurt', 'Nurnberg']
dfs Frankfurt-Nurnberg: ['Frankfurt', 'Mannheim', 'Karlsruhe', 'Augsburg', 'Munchen', 'Wurzburg', 'Erfurt', 'Nurnberg']
bfs Wurzburg-Kassel: Not found
dfs Wurzburg-Kassel: Not found


不再有bfs()和dfs()两个函数，我们将代码重构为使用单个traverse()函数。 traverse()
函数实际上是一个模板函数。它接受一个参数action，该参数是一个“知道”如何延伸路径的
函数。根据要使用的算法，我们可以传递extend_bfs_path()或extend_dfs_path()作为目
标动作。 

你也许会争论说，通过在traverse()内部添加一个条件来检测应该使用哪个遍历算法，也
能达到相同的结果。下面的代码展示了这个思路。 

In [5]:
BFS = 1
DFS = 2


def traverse(graph, start, end, algorithm):
    path = []
    visited = [start]
    while visited:
        current = visited.pop(0)
        if current not in path:
            path.append(current)
            if current == end:
                return (True, path)
            # 顶点不相连，则跳过
            if current not in graph:
                continue
        if algorithm == BFS:
            visited = extend_bfs_path(visited, graph[current])
        elif algorithm == DFS:
            visited = extend_dfs_path(visited, graph[current])
        else:
            raise ValueError("No such algorithm")
    return (False, path)


def extend_bfs_path(visited, current):
    return visited + current


def extend_dfs_path(visited, current):
    return current + visited


def main():
    graph = {
        'Frankfurt': ['Mannheim', 'Wurzburg', 'Kassel'],
        'Mannheim': ['Karlsruhe'],
        'Karlsruhe': ['Augsburg'],
        'Augsburg': ['Munchen'],
        'Wurzburg': ['Erfurt', 'Nurnberg'],
        'Nurnberg': ['Stuttgart', 'Munchen'],
        'Kassel': ['Munchen'],
        'Erfurt': [],
        'Stuttgart': [],
        'Munchen': []
    }

    bfs_path = traverse(graph, 'Frankfurt', 'Nurnberg', 1)
    dfs_path = traverse(graph, 'Frankfurt', 'Nurnberg', 2)
    print('bfs Frankfurt-Nurnberg: {}'.format(bfs_path[1] if bfs_path[0] else 'Not found'))
    print('dfs Frankfurt-Nurnberg: {}'.format(dfs_path[1] if dfs_path[0] else 'Not found'))

    bfs_nopath = traverse(graph, 'Wurzburg', 'Kassel', 1)
    print('bfs Wurzburg-Kassel: {}'.format(bfs_nopath[1] if bfs_nopath[0] else 'Not found'))
    dfs_nopath = traverse(graph, 'Wurzburg', 'Kassel', 2)
    print('dfs Wurzburg-Kassel: {}'.format(dfs_nopath[1] if dfs_nopath[0] else 'Not found'))


In [6]:
main()

bfs Frankfurt-Nurnberg: ['Frankfurt', 'Mannheim', 'Wurzburg', 'Kassel', 'Karlsruhe', 'Erfurt', 'Nurnberg']
dfs Frankfurt-Nurnberg: ['Frankfurt', 'Mannheim', 'Karlsruhe', 'Augsburg', 'Munchen', 'Wurzburg', 'Erfurt', 'Nurnberg']
bfs Wurzburg-Kassel: Not found
dfs Wurzburg-Kassel: Not found


我不喜欢这个方案，有以下几个原因。   
 它使得traverse()难以维护。如果添加第三种方式来延伸路径，就需要扩展traverse()
的代码，再添加一个条件来检测是否使用新的路径延伸动作。更好的方案是traverse()  
能发挥作用却好像根本不知道应该执行哪个action，因为这样在traverse()中不要求
什么特殊逻辑。 
 它仅对只有一行区别的算法有效。如果存在更多区别，那么与让本应归属action的具体
细节污染traverse()函数相比，创建一个新函数会好得多。      
 它使得traverse()更慢。这是因为每次traverse()执行时，都需要显式地检测应该执
行哪个遍历函数。    

### 现实生活的例子 
工人的日程，特别是对于同一个公司的工人而言，非常接近于模板设计模式。所有工人都遵
从或多或少相同的例行流程，但例行流程的某些特定部分区别又很大。情况如下图所示，该图由
www.sourcemaking.com提供（请参考网页［t.cn/RqrBWXo］）。图上展示的模板模式与使用Python
实现的模板模式的根本区别在于Python中不强制使用继承。仅在继承对实现有益时，我们才使用
它。如果没有实际益处，则可以忽略它，并使用命令和输入惯例。 

### 实现 
本节中，我们将实现一个横幅生成器。想法很简单，将一段文本发送给一个函数，该函数要
生成一个包含该文本的横幅。横幅有多种风格，比如点或虚线围绕文本。横幅生成器有一个默认
风格，但应该能够使用我们自己提供的风格。 
函数generate_banner()是我们的模板函数。它接受一个输入参数（msg，希望横幅包含
的文本）和一个可选参数（style，希望使用的风格）。默认风格是dots_style，我们马上就能
看到。generate_banner()以一个简单的头部和尾部来包装带样式的文本。实际上，这个头部
和尾部可以复杂得多，但在这里调用可以生成头部和尾部的函数来替代仅仅输出简单字符串也无
不可。 

In [8]:
from cowpy import cow


def dots_style(msg):
    msg = msg.capitalize()
    msg = '.' * 10 + msg + '.' * 10
    return msg


def admire_style(msg):
    msg = msg.upper()
    return '!'.join(msg)


def cow_style(msg):
    msg = cow.milk_random_cow(msg)
    return msg


def generate_banner(msg, style=dots_style):
    print('-- start of banner --')
    print(style(msg))
    print('-- end of banner --\n\n')


def main():
    msg = 'happy coding'
    [generate_banner(msg, style) for style in (dots_style, admire_style, cow_style)]

In [9]:
main()

-- start of banner --
..........Happy coding..........
-- end of banner --


-- start of banner --
H!A!P!P!Y! !C!O!D!I!N!G
-- end of banner --


-- start of banner --
 ______________ 
< happy coding >
 -------------- 
  \
   \   \_\_    _/_/
    \      \__/
           (oo)\_______
           (__)\       )\/\
            U    ||----w |
               ||     ||
-- end of banner --


